In [43]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
import numpy as np
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix
import time
from sklearn.model_selection import GridSearchCV
import joblib
import pickle

# импорт таблицы с матчами

In [77]:
df = pd.read_parquet("main.parquet")
df.drop(2276, inplace=True)
df = df.reset_index(drop=True)

In [78]:
df

,match_id,radiant_win,radiant_team,dire_team,avg_rank_tier
0,8546249912,False,"[74, 129, 84, 63, 48]","[90, 109, 119, 33, 62]",72
1,8546245917,True,"[101, 1, 119, 52, 96]","[129, 85, 7, 8, 100]",73
2,8546244800,False,"[70, 135, 123, 45, 137]","[102, 120, 5, 53, 84]",74
3,8546242707,False,"[33, 12, 30, 74, 2]","[65, 11, 100, 5, 36]",75
4,8546241701,True,"[16, 46, 63, 21, 52]","[2, 53, 34, 86, 41]",75
...,...,...,...,...,...
139069,8540008905,False,"[101, 34, 70, 23, 74]","[7, 27, 136, 79, 67]",65
139070,8540004805,True,"[12, 27, 34, 53, 2]","[75, 10, 52, 20, 129]",63
139071,8540003009,False,"[34, 3, 75, 7, 8]","[93, 17, 65, 128, 59]",75
139072,8539993900,True,"[40, 106, 105, 53, 64]","[39, 135, 35, 86, 7]",75


# Кодирование признаков - героев для RandomForest и LogisticRegression

In [79]:
# Собираем множество всех героев
all_heroes = sorted(set(np.concatenate(df['radiant_team'].values)).union(
                    set(np.concatenate(df['dire_team'].values))))

print("Всего героев:", len(all_heroes))
hero_matrix = np.zeros((len(df), len(all_heroes)), dtype=np.int8)

hero_index = {hero: i for i, hero in enumerate(all_heroes)}

for row_i, (rad, dire) in enumerate(zip(df['radiant_team'], df['dire_team'])):
    for h in rad:
        hero_matrix[row_i, hero_index[h]] = 1
    for h in dire:
        hero_matrix[row_i, hero_index[h]] = -1
hero_df = pd.DataFrame(hero_matrix, columns=[f"hero_{h}" for h in all_heroes])
df = pd.concat([df, hero_df], axis=1)

X = df[[col for col in df.columns if col.startswith("hero_")] + ["avg_rank_tier"]]
y = df["radiant_win"]

df = df.loc[:, ~df.columns.duplicated()]

[col for col in df.columns if df.columns.tolist().count(col) > 1]

df.loc[0, ['hero_106','hero_105','hero_2','hero_131']]

Всего героев: 126


hero_106    0
hero_105    0
hero_2      0
hero_131    0
Name: 0, dtype: object

In [82]:
df.head()

,match_id,radiant_win,radiant_team,dire_team,avg_rank_tier,hero_1,hero_2,hero_3,hero_4,hero_5,...,hero_123,hero_126,hero_128,hero_129,hero_131,hero_135,hero_136,hero_137,hero_138,hero_145
0,8546249912,False,"[74, 129, 84, 63, 48]","[90, 109, 119, 33, 62]",72,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,8546245917,True,"[101, 1, 119, 52, 96]","[129, 85, 7, 8, 100]",73,1,0,0,0,0,...,0,0,0,-1,0,0,0,0,0,0
2,8546244800,False,"[70, 135, 123, 45, 137]","[102, 120, 5, 53, 84]",74,0,0,0,0,-1,...,1,0,0,0,0,1,0,1,0,0
3,8546242707,False,"[33, 12, 30, 74, 2]","[65, 11, 100, 5, 36]",75,0,1,0,0,-1,...,0,0,0,0,0,0,0,0,0,0
4,8546241701,True,"[16, 46, 63, 21, 52]","[2, 53, 34, 86, 41]",75,0,-1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
df.to_parquet('matches_for_learn2.parquet', engine='pyarrow')

# Кодирование признаков - героев для CatBoost  
Используется ohe по слотам, так как Catboost умеет ловить взаимодействия лучше, если дать ему “герой в слоте”, а не просто "герой есть"

In [110]:
df66 = pd.read_parquet("main1.parquet")
#df1.drop(2276, inplace=True)
df66 = df66.reset_index(drop=True)
df66

,match_id,radiant_win,radiant_team,dire_team,avg_rank_tier
0,8546249912,False,"[74, 129, 84, 63, 48]","[90, 109, 119, 33, 62]",72
1,8546245917,True,"[101, 1, 119, 52, 96]","[129, 85, 7, 8, 100]",73
2,8546244800,False,"[70, 135, 123, 45, 137]","[102, 120, 5, 53, 84]",74
3,8546242707,False,"[33, 12, 30, 74, 2]","[65, 11, 100, 5, 36]",75
4,8546241701,True,"[16, 46, 63, 21, 52]","[2, 53, 34, 86, 41]",75
...,...,...,...,...,...
139069,8540008905,False,"[101, 34, 70, 23, 74]","[7, 27, 136, 79, 67]",65
139070,8540004805,True,"[12, 27, 34, 53, 2]","[75, 10, 52, 20, 129]",63
139071,8540003009,False,"[34, 3, 75, 7, 8]","[93, 17, 65, 128, 59]",75
139072,8539993900,True,"[40, 106, 105, 53, 64]","[39, 135, 35, 86, 7]",75


In [111]:
all_heroes = set()

for team in ['radiant_team', 'dire_team']:
    df66[team] = df66[team].apply(lambda x: list(x))  # убедимся, что это список
    df66[team].apply(all_heroes.update)

all_heroes = sorted(list(all_heroes))

In [118]:
def prepare_features(df):
    # --- 1. Убедимся, что radiant_team и dire_team — это списки ---
    df = df.copy()
    df['radiant_team'] = df['radiant_team'].apply(lambda x: list(x))
    df['dire_team'] = df['dire_team'].apply(lambda x: list(x))
    
    # --- 2. Собираем список всех героев ---
    all_heroes = set()
    df['radiant_team'].apply(all_heroes.update)
    df['dire_team'].apply(all_heroes.update)
    all_heroes = sorted(all_heroes)

    # --- 3. Подготовим массив будущих признаков ---
    feature_columns = []

    # создаём пустой датафрейм под OHE признаки
    slot_features = pd.DataFrame(
        np.zeros((len(df), 10 * len(all_heroes))),  # 10 позиций = 5 Radiant + 5 Dire
        index=df.index
    )

    col_idx = 0
    for slot in range(5):
        for hero in all_heroes:
            col_name_r = f"radiant_slot{slot+1}_h{hero}"
            col_name_d = f"dire_slot{slot+1}_h{hero}"

            slot_features[col_idx] = df['radiant_team'].apply(lambda x, h=hero, s=slot: 1 if x[s] == h else 0)
            slot_features.rename(columns={col_idx: col_name_r}, inplace=True)
            feature_columns.append(col_name_r)
            col_idx += 1

            slot_features[col_idx] = df['dire_team'].apply(lambda x, h=hero, s=slot: 1 if x[s] == h else 0)
            slot_features.rename(columns={col_idx: col_name_d}, inplace=True)
            feature_columns.append(col_name_d)
            col_idx += 1

    # --- 4. Добавляем признаки к исходному датасету ---
    df = pd.concat([df, slot_features], axis=1)

    # --- 5. Удаляем исходные списочные признаки ---
    #df = df.drop(columns=['radiant_team', 'dire_team'])

    return df

In [119]:
df1 = prepare_features(df66)

In [120]:
df1

,match_id,radiant_win,radiant_team,dire_team,avg_rank_tier,radiant_slot1_h1,dire_slot1_h1,radiant_slot1_h2,dire_slot1_h2,radiant_slot1_h3,...,radiant_slot5_h135,dire_slot5_h135,radiant_slot5_h136,dire_slot5_h136,radiant_slot5_h137,dire_slot5_h137,radiant_slot5_h138,dire_slot5_h138,radiant_slot5_h145,dire_slot5_h145
0,8546249912,False,"[74, 129, 84, 63, 48]","[90, 109, 119, 33, 62]",72,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8546245917,True,"[101, 1, 119, 52, 96]","[129, 85, 7, 8, 100]",73,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8546244800,False,"[70, 135, 123, 45, 137]","[102, 120, 5, 53, 84]",74,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,8546242707,False,"[33, 12, 30, 74, 2]","[65, 11, 100, 5, 36]",75,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8546241701,True,"[16, 46, 63, 21, 52]","[2, 53, 34, 86, 41]",75,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139069,8540008905,False,"[101, 34, 70, 23, 74]","[7, 27, 136, 79, 67]",65,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
139070,8540004805,True,"[12, 27, 34, 53, 2]","[75, 10, 52, 20, 129]",63,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
139071,8540003009,False,"[34, 3, 75, 7, 8]","[93, 17, 65, 128, 59]",75,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
139072,8539993900,True,"[40, 106, 105, 53, 64]","[39, 135, 35, 86, 7]",75,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [114]:
df1.to_parquet('matches_for_catboost66.parquet', engine='pyarrow')

In [69]:
df = pd.read_parquet('matches_for_catboost2.parquet')
df

,match_id,radiant_win,avg_rank_tier,radiant_slot1_h1,dire_slot1_h1,radiant_slot1_h2,dire_slot1_h2,radiant_slot1_h3,dire_slot1_h3,radiant_slot1_h4,...,radiant_slot5_h135,dire_slot5_h135,radiant_slot5_h136,dire_slot5_h136,radiant_slot5_h137,dire_slot5_h137,radiant_slot5_h138,dire_slot5_h138,radiant_slot5_h145,dire_slot5_h145
0,8546249912,False,72,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8546245917,True,73,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8546244800,False,74,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,8546242707,False,75,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8546241701,True,75,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139070,8540008905,False,65,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
139071,8540004805,True,63,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
139072,8540003009,False,75,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
139073,8539993900,True,75,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# новое кодирование, добавление признаков

In [115]:
def add_simple_features(df):
    df = df.copy()
    
    # 1. Размеры команд (на случай неполных данных)
    df['radiant_size'] = df['radiant_team'].apply(len)
    df['dire_size'] = df['dire_team'].apply(len)
    
    # 2. Простые статистики по героям
    all_heroes = list(set([h for team in df['radiant_team'] for h in team] + 
                          [h for team in df['dire_team'] for h in team]))
    
    # 3. Частота героев в датасете
    hero_freq = {}
    for team in df['radiant_team']:
        for hero in team:
            hero_freq[hero] = hero_freq.get(hero, 0) + 1
    for team in df['dire_team']:
        for hero in team:
            hero_freq[hero] = hero_freq.get(hero, 0) + 1
    
    df['radiant_avg_popularity'] = df['radiant_team'].apply(
        lambda team: np.mean([hero_freq.get(h, 0) for h in team])
    )
    df['dire_avg_popularity'] = df['dire_team'].apply(
        lambda team: np.mean([hero_freq.get(h, 0) for h in team])
    )
    
    return df

# df_simple = add_simple_features(df)

def target_encode_heroes(df, target='radiant_win', n_folds=5):
    df = df.copy()
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    # Создаем колонки для encoding
    df['radiant_hero_encoding'] = 0.0
    df['dire_hero_encoding'] = 0.0
    
    for train_idx, val_idx in kf.split(df):
        train_df = df.iloc[train_idx]
        
        # Считаем винрейты героев на train
        hero_win_rates = {}
        for idx, row in train_df.iterrows():
            for hero in row['radiant_team']:
                hero_win_rates[hero] = hero_win_rates.get(hero, []) + [row[target]]
            for hero in row['dire_team']:
                hero_win_rates[hero] = hero_win_rates.get(hero, []) + [1 - row[target]]
        
        # Усредняем
        hero_encoding = {hero: np.mean(wins) for hero, wins in hero_win_rates.items()}
        
        # Применяем к validation set
        for idx in val_idx:
            row = df.iloc[idx]
            df.at[idx, 'radiant_hero_encoding'] = np.mean([hero_encoding.get(h, 0.5) for h in row['radiant_team']])
            df.at[idx, 'dire_hero_encoding'] = np.mean([hero_encoding.get(h, 0.5) for h in row['dire_team']])
    
    return df

#df_encoded = target_encode_heroes(df)

def calculate_basic_synergy(df):
    """Простой расчет синергий на основе частоты совместных побед"""
    from collections import defaultdict
    
    pair_stats = defaultdict(lambda: {'wins': 0, 'games': 0})
    
    for _, row in df.iterrows():
        radiant_team = row['radiant_team']
        dire_team = row['dire_team']
        radiant_win = row['radiant_win']
        
        # Синергии в Radiant
        for i in range(len(radiant_team)):
            for j in range(i + 1, len(radiant_team)):
                pair = tuple(sorted([radiant_team[i], radiant_team[j]]))
                pair_stats[pair]['games'] += 1
                pair_stats[pair]['wins'] += radiant_win
        
        # Синергии в Dire
        for i in range(len(dire_team)):
            for j in range(i + 1, len(dire_team)):
                pair = tuple(sorted([dire_team[i], dire_team[j]]))
                pair_stats[pair]['games'] += 1
                pair_stats[pair]['wins'] += (1 - radiant_win)
    
    # Вычисляем винрейты пар
    pair_win_rates = {}
    for pair, stats in pair_stats.items():
        if stats['games'] >= 5:  # Минимальное количество игр
            pair_win_rates[pair] = stats['wins'] / stats['games']
        else:
            pair_win_rates[pair] = 0.5  # нейтральное значение
    
    return pair_win_rates

def add_synergy_features(df, pair_win_rates):
    df = df.copy()
    
    def team_synergy(team):
        synergies = []
        for i in range(len(team)):
            for j in range(i + 1, len(team)):
                pair = tuple(sorted([team[i], team[j]]))
                synergies.append(pair_win_rates.get(pair, 0.5))
        return np.mean(synergies) if synergies else 0.5
    
    df['radiant_synergy'] = df['radiant_team'].apply(team_synergy)
    df['dire_synergy'] = df['dire_team'].apply(team_synergy)
    df['synergy_diff'] = df['radiant_synergy'] - df['dire_synergy']
    
    return df

# Применяем
#pair_win_rates = calculate_basic_synergy(df)
#df_with_synergy = add_synergy_features(df, pair_win_rates)

In [117]:
def create_all_features(df):
    # Копия исходного датафрейма
    df_features = df.copy()
    
    # Сохраняем команды
    radiant_teams = df_features['radiant_team'].copy()
    dire_teams = df_features['dire_team'].copy()
    
    # ---- 1. Простые фичи ----
    df_features = add_simple_features(df_features)
    
    # ---- 2. Target encoding ----
    df_features = target_encode_heroes(df_features)
    
    # ---- 3. Синергии ----
    pair_win_rates = calculate_basic_synergy(df_features)
    df_features = add_synergy_features(df_features, pair_win_rates)
    
    # ---- 4. Удаляем исходные списки героев ----
    df_features = df_features.drop(['radiant_team', 'dire_team'], axis=1)
    
    return df_features

In [ ]:
#отеляем признаки

In [123]:
X = create_all_features(df)

In [134]:
y = df['radiant_win'].astype(int)

In [153]:
X1 = create_all_features(df1)

In [155]:
X1 = X1.drop(columns=['match_id', 'radiant_win'])

In [10]:
#обучение

In [ ]:
#train test

In [136]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
print(X_train.shape, X_test.shape)

(111259, 136) (27815, 136)


In [159]:
X1_train, X1_test, y1_train, y1_test = train_test_split(
    X1, y, test_size=0.2, stratify=y, random_state=42
)
print(X1_train.shape, X1_test.shape)

(111259, 1270) (27815, 1270)


In [139]:
logreg = LogisticRegression(max_iter=2000, n_jobs=-1)
logreg.fit(X_train, y_train)

LogisticRegression(max_iter=2000, n_jobs=-1)

In [140]:
rf = RandomForestClassifier(
    n_estimators=800,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2
)
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=20, min_samples_leaf=2, min_samples_split=5,
                       n_estimators=800)

In [143]:
def evaluate(model, name):
    pred = model.predict(X_test)
    prob = model.predict_proba(X_test)[:, 1]
    print(f"{name}")
    print("Accuracy:", round(accuracy_score(y_test, pred), 4))
    print("ROC-AUC:", round(roc_auc_score(y_test, prob), 4))
    print()
evaluate(logreg, "Logistic Regression")
evaluate(rf, "Random Forest")

Logistic Regression
Accuracy: 0.5722
ROC-AUC: 0.5947

Random Forest
Accuracy: 0.6041
ROC-AUC: 0.6392



In [160]:
model = CatBoostClassifier(
    iterations=3000,
    depth=7,
    learning_rate=0.02,
    loss_function='Logloss',
    eval_metric='AUC',
    random_state=42,
    verbose=False
)
model.fit(X1_train, y_train, eval_set=(X1_test, y_test))

In [161]:
pred_proba = model.predict_proba(X1_test)[:, 1]
pred = model.predict(X1_test)
print("Accuracy:", accuracy_score(y_test, pred))
print("ROC-AUC:", roc_auc_score(y_test, pred_proba))

Accuracy: 0.6128348013661693
ROC-AUC: 0.6523246073128711
